In [2]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_rows',20000)
dataset_df = pd.read_csv('/home/chang/domain_intent/domain_dataset.csv')

In [3]:
dataset_df.columns = ['verse', 'domain']
print(dataset_df)

                                                   verse               domain
0      Then all the people of Judah took Uzziah, who ...                bible
1      Uzziah was sixteen years old when he became ki...                bible
2      The Ammonites gave offerings to Uzziah: and ne...                bible
3      Uzziah made towers in Jerusalem, at the doorwa...                bible
4      In addition, Uzziah had an army of fighting-me...                bible
...                                                  ...                  ...
42879  what very will the afghan good be like this we...  NotPlayMusicOrVideo
42880             rate this textbook two out of 6 points  NotPlayMusicOrVideo
42881  zero that textbook includes two point this sco...  NotPlayMusicOrVideo
42882  reduced wins this textbook followed tries out ...  NotPlayMusicOrVideo
42883  rate in this critique two pass out views only ...  NotPlayMusicOrVideo

[42884 rows x 2 columns]


In [5]:
# condition_list = [
#     (dataset_df['domain'] == 'bible'),
#     (dataset_df['domain'] == 'PlayMusicOrVideo'),
#     (dataset_df['domain'] == 'NotPlayMusicOrVideo')
# ]
# choice_list = [int(0),int(1),int(2)]
#dataset_df['label'] = [0 if y == 'not bible' else 1 for y in dataset_df['domain']]

def func(x):
    if x == 'bible':
        return 0
    elif x == 'PlayMusicOrVideo':
        return 1
    else:
        return 2
# dataset_df['label'] = [0 if y == 'bible' for y in dataset_df['domain']]
# dataset_df['label'] = np.select(condition_list,choice_list,default='none')
dataset_df['label'] = dataset_df['domain'].apply(lambda x : func(x))
print(dataset_df)

                                                   verse               domain  \
0      Then all the people of Judah took Uzziah, who ...                bible   
1      Uzziah was sixteen years old when he became ki...                bible   
2      The Ammonites gave offerings to Uzziah: and ne...                bible   
3      Uzziah made towers in Jerusalem, at the doorwa...                bible   
4      In addition, Uzziah had an army of fighting-me...                bible   
...                                                  ...                  ...   
42879  what very will the afghan good be like this we...  NotPlayMusicOrVideo   
42880             rate this textbook two out of 6 points  NotPlayMusicOrVideo   
42881  zero that textbook includes two point this sco...  NotPlayMusicOrVideo   
42882  reduced wins this textbook followed tries out ...  NotPlayMusicOrVideo   
42883  rate in this critique two pass out views only ...  NotPlayMusicOrVideo   

       label  
0          0

In [7]:
type(dataset_df['label'][0])

numpy.int64

In [8]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertConfig
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import numpy as np
from sklearn.metrics import f1_score
import pickle
from torch.nn.utils.rnn import pad_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch.nn.functional as F

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(dataset_df.index.values, 
                                                  dataset_df.label.values, 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=dataset_df.label.values)

dataset_df['data_type'] = ['not_set']*dataset_df.shape[0]

dataset_df.loc[X_train, 'data_type'] = 'train'
dataset_df.loc[X_val, 'data_type'] = 'val'

print(dataset_df.groupby(['domain', 'label', 'data_type']).count())

                                     verse
domain              label data_type       
NotPlayMusicOrVideo 2     train      10452
                          val         4480
PlayMusicOrVideo    1     train      10449
                          val         4479
bible               0     train       9117
                          val         3907


In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    dataset_df[dataset_df.data_type=='train'].verse.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    dataset_df[dataset_df.data_type=='val'].verse.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(dataset_df[dataset_df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(dataset_df[dataset_df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/chang/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2251: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
print(dataset_df[dataset_df.data_type=='train'].label.values[0])

0


In [11]:
label_dict = { 'bible': 0, 'PlayMusicOrVideo': 1 ,'NotPlayMusicOrVideo': 2}
print(label_dict)

{'bible': 0, 'PlayMusicOrVideo': 1, 'NotPlayMusicOrVideo': 2}


In [12]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False).cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)


In [14]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/home/chang/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [16]:
device = torch.device('cuda')

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    
    torch.save(model.state_dict(), f'/home/chang/domain_intent/finetuned_BERT_epoch_{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/939 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.10879718167340349
Validation loss: 0.047199962315324084
F1 Score (Weighted): 0.9863205933481125


Epoch 2:   0%|          | 0/939 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.024022240309477152
Validation loss: 0.04890045393286498
F1 Score (Weighted): 0.9889642577602874


...


Epoch 3
Training loss: 0.01044654968608812
Validation loss: 0.0511478285286642
F1 Score (Weighted): 0.9891949000014741


Epoch 4:   0%|          | 0/939 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.003983536133969079
Validation loss: 0.04715476500744401
F1 Score (Weighted): 0.9912166346358416


Epoch 5:   0%|          | 0/939 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.0017698824547346002
Validation loss: 0.04920106850379674
F1 Score (Weighted): 0.9912166984222613


In [19]:
# pad_sent = pad_sequences([embedding_tokens], maxlen=256, dtype='long', truncating='post', padding='post')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def Bert_tokenizer(sent):
    
    bert_sent = '[CLS] ' + sent + ' [SEP]'
    tokenized_text = tokenizer.tokenize(bert_sent)
    
    embedding_tokens = []
    for text in tokenized_text:
        encoded_sent = tokenizer.convert_tokens_to_ids(text)
        embedding_tokens.append(encoded_sent)
    
    return embedding_tokens

def pad_sentences(sent, max_len=256):
    embedding_sent = pad_sequences([sent], maxlen=256, dtype='long', truncating='post', padding='post')
    return embedding_sent

def create_masks(sent):
    masks = []
    for token in sent:
        mask = [float(t>0) for t in token]
        masks.append(mask)
    return masks

def convert_data(sent):
    tokenized_sent = Bert_tokenizer(sent)
    padded_sent = pad_sentences(tokenized_sent)
    
    masks_sent = create_masks(padded_sent)
    
    inputs = torch.tensor(padded_sent)
    masks = torch.tensor(masks_sent)
    
    return inputs, masks

def test_sentences(sent):
    
    model.eval()
    
    print('test_sentences')
    inputs, masks = convert_data(sent)
    print('convert_data')
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
    

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    logits = F.softmax(outputs[0], dim=1)
    logits = logits.cpu().numpy()
    print(logits)
    print(type(logits))
    print(logits.shape)
    pred = np.argmax(logits)
    print(logits[0][pred]*100)
    if pred == 0:
        result = "bible"
    elif pred == 1:
        result = "PlayMusicOrVideo"
    else:
        result = "general QA"
        
    return result

In [ ]:
sent = "How's the weather?"
print(test_sentences(sent))

In [25]:
import time
start = time.time()

sent = "play a video of Chansung Jung and Volkanovsky"
print(test_sentences(sent))
# label = test_sentences(sent)
print("time :", time.time() - start)

test_sentences
convert_data
[[3.3054344e-05 9.9975544e-01 2.1149062e-04]]
<class 'numpy.ndarray'>
(1, 3)
99.97554421424866
PlayMusicOrVideo
time : 0.017194509506225586


In [23]:
start = time.time()

sent = "what is machine learning?"
print(test_sentences(sent))
# label = test_sentences(sent)
print("time :", time.time() - start)

test_sentences
convert_data
[[1.7874794e-03 4.4552161e-04 9.9776697e-01]]
<class 'numpy.ndarray'>
(1, 3)
99.77669715881348
general QA
time : 0.037783145904541016


In [38]:
start = time.time()

sent = "What does the word Israel mean?"
print(test_sentences(sent))
# label = test_sentences(sent)
print("time :", time.time() - start)

test_sentences
convert_data
[[0.03219715 0.00134538 0.9664574 ]]
<class 'numpy.ndarray'>
(1, 3)
96.64574265480042
general QA
time : 0.013893604278564453
